# Projeto Estágio supervisionado - ETL

O objetivo desde notbook é Desenvolver o processo de Inserção dos dados nas tabelas já existentes no Data Warehouse, conforme o diagrama de Entidade-Relacionamento (DER) e o Script schema.sql

In [1]:
# Imports

import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import urllib.parse # Resolver problema com a senha do banco CARACTER ESPECIAL
import os

### Conexão com o Banco de Dados

Nesta etapa, vamos criar uma conexão com o banco de dados MySQL utilizando o SQLAlchemy. 


In [2]:
PASTA_DADOS = 'Data'

# Dados de conexão

db_user = 'root'
db_password = '01611478Marlin@'
db_host = 'localhost'
db_port = '3306'
db_name = 'projetopetroleo'

senha_codificada = urllib.parse.quote_plus(db_password)

# Criando uma conexão com o banco de dados
try: 
    con_str = f"mysql+mysqlconnector://{db_user}:{senha_codificada}@{db_host}:{db_port}/{db_name}" 
    engine = create_engine(con_str)
    print('Conectado ao banco de dados com sucesso!')
except Exception as e:
    print(f'Erro ao conectar ao banco de dados: {e}')

Conectado ao banco de dados com sucesso!


## `dim_calendario`
### Geração dos Dados do Calendário

Nesta etapa, será criado um DataFrame do Pandas contendo todos os dias de 1990 a 2030. Em seguida, será extraído e criada todas as colunas de enriquecimento que definimos no nosso modelo (ano, mês, trimestre, etc.), incluindo a nossa chave primária no formato `AAAAMMDD`.

In [3]:
# Definindo o período de datas
data_inicio = '1990-01-01'
data_fim = '2030-12-31'

datas = pd.date_range(start=data_inicio, end=data_fim)
df_calendario = pd.DataFrame(datas, columns=['data_completa'])

# Criando a PK (AAAAMMDD)
df_calendario['id_calendario'] = df_calendario['data_completa'].dt.strftime('%Y%m%d').astype(int)

# Extraindo e enriquecendo os atributos da data
df_calendario['ano'] = df_calendario['data_completa'].dt.year
df_calendario['mes_numero'] = df_calendario['data_completa'].dt.month
df_calendario['dia_numero'] = df_calendario['data_completa'].dt.day
df_calendario['trimestre_numero'] = df_calendario['data_completa'].dt.quarter
df_calendario['semestre_numero'] = (df_calendario['data_completa'].dt.quarter + 1) // 2

# Mapeando nomes em português
mapa_meses = {1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'}
mapa_dias_semana = {0: 'Segunda-feira', 1: 'Terça-feira', 2: 'Quarta-feira', 3: 'Quinta-feira', 4: 'Sexta-feira', 5: 'Sábado', 6: 'Domingo'}

df_calendario['mes_nome'] = df_calendario['mes_numero'].map(mapa_meses)
df_calendario['trimestre_nome'] = 'T' + df_calendario['trimestre_numero'].astype(str)
df_calendario['semestre_nome'] = 'S' + df_calendario['semestre_numero'].astype(str)
df_calendario['dia_da_semana'] = df_calendario['data_completa'].dt.dayofweek.map(mapa_dias_semana)

# Organizando as colunas na ordem correta da tabela
ordem_colunas = [
    'id_calendario', 'data_completa', 'ano', 'mes_numero', 'mes_nome',
    'dia_numero', 'trimestre_numero', 'trimestre_nome', 'semestre_numero',
    'semestre_nome', 'dia_da_semana'
]
df_calendario = df_calendario[ordem_colunas]

print("DataFrame da dim_calendario gerado com sucesso.")

DataFrame da dim_calendario gerado com sucesso.


### Carregando os Dados para o Banco de Dados

Com o DataFrame pronto e formatado exatamente como a nossa tabela `dim_calendario` no MySQL, agora podemos fazer a carga dos dados. Usaremos o método `.to_sql()` do Pandas, que é otimizado para inserção em massa.

In [4]:
# Carregando o DataFrame para a tabela SQL
try:
    df_calendario.to_sql('dim_calendario', con=engine, if_exists='append', index=False)
    print(f"{len(df_calendario)} registros carregados com sucesso na tabela 'dim_calendario'!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

14975 registros carregados com sucesso na tabela 'dim_calendario'!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [5]:
# Visualizando o resultado
display(df_calendario.head())
display(df_calendario.tail())
display(df_calendario.info())

,id_calendario,data_completa,ano,mes_numero,mes_nome,dia_numero,trimestre_numero,trimestre_nome,semestre_numero,semestre_nome,dia_da_semana
0,19900101,1990-01-01,1990,1,Janeiro,1,1,T1,1,S1,Segunda-feira
1,19900102,1990-01-02,1990,1,Janeiro,2,1,T1,1,S1,Terça-feira
2,19900103,1990-01-03,1990,1,Janeiro,3,1,T1,1,S1,Quarta-feira
3,19900104,1990-01-04,1990,1,Janeiro,4,1,T1,1,S1,Quinta-feira
4,19900105,1990-01-05,1990,1,Janeiro,5,1,T1,1,S1,Sexta-feira


,id_calendario,data_completa,ano,mes_numero,mes_nome,dia_numero,trimestre_numero,trimestre_nome,semestre_numero,semestre_nome,dia_da_semana
14970,20301227,2030-12-27,2030,12,Dezembro,27,4,T4,2,S2,Sexta-feira
14971,20301228,2030-12-28,2030,12,Dezembro,28,4,T4,2,S2,Sábado
14972,20301229,2030-12-29,2030,12,Dezembro,29,4,T4,2,S2,Domingo
14973,20301230,2030-12-30,2030,12,Dezembro,30,4,T4,2,S2,Segunda-feira
14974,20301231,2030-12-31,2030,12,Dezembro,31,4,T4,2,S2,Terça-feira


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14975 entries, 0 to 14974
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_calendario     14975 non-null  int64         
 1   data_completa     14975 non-null  datetime64[ns]
 2   ano               14975 non-null  int32         
 3   mes_numero        14975 non-null  int32         
 4   mes_nome          14975 non-null  object        
 5   dia_numero        14975 non-null  int32         
 6   trimestre_numero  14975 non-null  int32         
 7   trimestre_nome    14975 non-null  object        
 8   semestre_numero   14975 non-null  int32         
 9   semestre_nome     14975 non-null  object        
 10  dia_da_semana     14975 non-null  object        
dtypes: datetime64[ns](1), int32(5), int64(1), object(4)
memory usage: 994.6+ KB


None

## `dim_localizacao`
Neste etapa, vamos popular a tabela `dim_localizacao` extraindo, limpando e unificando dados de localização de múltiplos arquivos de origem.

In [6]:

# Lista de TODOS os arquivos que vamos processar para ter uma dimensão completa
arquivos_com_local = [
    'producao-petroleo-m3-1997-2025.csv',
    'processamento-petroleo-m3-1990-2025.csv',
   'ca-2004-01.csv', 'ca-2004-02.csv',
    'ca-2010-01.csv', 'ca-2010-02.csv',
    'ca-2016-01.csv', 'ca-2016-02.csv',
    'ca-2020-01.csv', 'ca-2020-02.csv',
    'ca-2022-01.csv', 'ca-2022-02.csv',
    'ca-2025-01.csv'
]

lista_de_dfs_locais = []

# Dicionário de mapeamento de colunas
mapa_colunas = {
    'GRANDE REGIÃO': 'regiao', 'UNIDADE DA FEDERAÇÃO': 'estado',
    'Estado - Sigla': 'uf_sigla', 'Regiao - Sigla': 'regiao_sigla', # Será descartada depois, mas ajuda a unificar
    'Municipio': 'municipio', 'Bairro': 'bairro', 'Cep': 'cep',
    'Nome da Rua': 'nome_rua', 'Numero Rua': 'numero_rua'
}

# Loop principal pelos arquivos
for nome_arquivo in arquivos_com_local:
    caminho_completo = os.path.join(PASTA_DADOS, nome_arquivo)
    print(f"Processando arquivo: {nome_arquivo}...")
    
    # Lendo apenas o cabeçalho para ver quais colunas de localização existem no arquivo
    colunas_originais_no_arquivo = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig', nrows=0).columns
    colunas_para_ler = [col for col in colunas_originais_no_arquivo if col in mapa_colunas.keys()]
    
    if not colunas_para_ler:
        print("Nenhuma coluna de localização encontrada. Pulando.")
        continue
        
    # Lendo arquivo em pedaços (chunks) usando apenas as colunas necessárias
    chunk_iterator = pd.read_csv(
        caminho_completo, 
        sep=';', 
        encoding='utf-8-sig',
        usecols=colunas_para_ler,      # OTIMIZAÇÃO 1
        chunksize=50000,               # OTIMIZAÇÃO 2
        low_memory=False
    )
    
    for chunk in chunk_iterator:
        chunk.rename(columns=mapa_colunas, inplace=True)
        lista_de_dfs_locais.append(chunk)

# Unificando todos os pedaços
df_locais_completo = pd.concat(lista_de_dfs_locais, ignore_index=True)
print("\nTodos os arquivos foram lidos e unificados.")

# Limpando o CEP para conter apenas números e padronizando S/N
chunk['cep'] = chunk['cep'].str.replace(r'\D', '', regex=True)
chunk['numero_rua'] = chunk['numero_rua'].astype(str).str.replace(r'(?i)s\/?n', 'S/N', regex=True)

# Renomeia todas as colunas
df_locais_completo.rename(columns=mapa_colunas, inplace=True)


print("Iniciando enriquecimento de dados de localização...")

# Criando os dicionários de mapeamento COMPLETOS
uf_para_estado = {
    'AC': 'ACRE', 'AL': 'ALAGOAS', 'AP': 'AMAPÁ', 'AM': 'AMAZONAS', 'BA': 'BAHIA', 
    'CE': 'CEARÁ', 'DF': 'DISTRITO FEDERAL', 'ES': 'ESPÍRITO SANTO', 'GO': 'GOIÁS', 
    'MA': 'MARANHÃO', 'MT': 'MATO GROSSO', 'MS': 'MATO GROSSO DO SUL', 'MG': 'MINAS GERAIS', 
    'PA': 'PARÁ', 'PB': 'PARAÍBA', 'PR': 'PARANÁ', 'PE': 'PERNAMBUCO', 'PI': 'PIAUÍ', 
    'RJ': 'RIO DE JANEIRO', 'RN': 'RIO GRANDE DO NORTE', 'RS': 'RIO GRANDE DO SUL', 
    'RO': 'RONDÔNIA', 'RR': 'RORAIMA', 'SC': 'SANTA CATARINA', 'SP': 'SÃO PAULO', 
    'SE': 'SERGIPE', 'TO': 'TOCANTINS'
}
estado_para_uf = {v: k for k, v in uf_para_estado.items()} # Mapa reverso automático

uf_para_regiao = {
    'AC': 'REGIÃO NORTE', 'AP': 'REGIÃO NORTE', 'AM': 'REGIÃO NORTE', 'PA': 'REGIÃO NORTE', 'RO': 'REGIÃO NORTE', 'RR': 'REGIÃO NORTE', 'TO': 'REGIÃO NORTE',
    'AL': 'REGIÃO NORDESTE', 'BA': 'REGIÃO NORDESTE', 'CE': 'REGIÃO NORDESTE', 'MA': 'REGIÃO NORDESTE', 'PB': 'REGIÃO NORDESTE', 'PE': 'REGIÃO NORDESTE', 'PI': 'REGIÃO NORDESTE', 'RN': 'REGIÃO NORDESTE', 'SE': 'REGIÃO NORDESTE',
    'DF': 'REGIÃO CENTRO-OESTE', 'GO': 'REGIÃO CENTRO-OESTE', 'MT': 'REGIÃO CENTRO-OESTE', 'MS': 'REGIÃO CENTRO-OESTE',
    'ES': 'REGIÃO SUDESTE', 'MG': 'REGIÃO SUDESTE', 'RJ': 'REGIÃO SUDESTE', 'SP': 'REGIÃO SUDESTE',
    'PR': 'REGIÃO SUL', 'RS': 'REGIÃO SUL', 'SC': 'REGIÃO SUL'
}

# Preenchendo os valores nulos em uma ordem lógica
# garantindo que as colunas 'estado' e 'uf_sigla' preencham uma à outra.
df_locais_completo['uf_sigla'] = df_locais_completo['uf_sigla'].fillna(df_locais_completo['estado'].str.upper().map(estado_para_uf))
df_locais_completo['estado'] = df_locais_completo['estado'].fillna(df_locais_completo['uf_sigla'].str.upper().map(uf_para_estado))
df_locais_completo['regiao'] = df_locais_completo['regiao'].fillna(df_locais_completo['uf_sigla'].str.upper().map(uf_para_regiao))
print("Enriquecimento concluído.")

# Agrupando por estado e município para criar uma linha única para cada localidade

print("Consolidando localidades únicas...")

# Usando .first() para pegar o primeiro valor não-nulo de cada coluna para aquele grupo
# Isso efetivamente "mescla" as informações das diferentes fontes
df_localizacao_final = df_locais_completo.groupby(['estado', 'uf_sigla', 'municipio']).first().reset_index()

# Garantindo que todas as colunas da dimensão final existam
colunas_dimensao = [
    'municipio', 'estado', 'uf_sigla', 'regiao', 
    'bairro', 'cep', 'nome_rua', 'numero_rua'
]
for col in colunas_dimensao:
    if col not in df_localizacao_final.columns:
        df_localizacao_final[col] = None
        
df_localizacao_final = df_localizacao_final[colunas_dimensao]
print("Limpeza e consolidação final concluídas.")

Processando arquivo: producao-petroleo-m3-1997-2025.csv...
Processando arquivo: processamento-petroleo-m3-1990-2025.csv...
Processando arquivo: ca-2004-01.csv...
Processando arquivo: ca-2004-02.csv...
Processando arquivo: ca-2010-01.csv...
Processando arquivo: ca-2010-02.csv...
Processando arquivo: ca-2016-01.csv...
Processando arquivo: ca-2016-02.csv...
Processando arquivo: ca-2020-01.csv...
Processando arquivo: ca-2020-02.csv...
Processando arquivo: ca-2022-01.csv...
Processando arquivo: ca-2022-02.csv...
Processando arquivo: ca-2025-01.csv...

Todos os arquivos foram lidos e unificados.
Iniciando enriquecimento de dados de localização...
Enriquecimento concluído.
Consolidando localidades únicas...
Limpeza e consolidação final concluídas.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_localizacao_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `dim_localizacao` que criamos no MySQL.

In [7]:
# Carregando os dados para a tabela dim_localizacao
print("Iniciando a carga para a tabela 'dim_localizacao'...")
try:
    df_localizacao_final.to_sql('dim_localizacao', con=engine, if_exists='append', index=False)
    print(f"{len(df_localizacao_final)} registros únicos de localização carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

Iniciando a carga para a tabela 'dim_localizacao'...
670 registros únicos de localização carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [8]:
print(f"Total de {len(df_localizacao_final)} localidades únicas foram encontradas e carregadas.")
display(df_localizacao_final.head())
display(df_localizacao_final.tail())
display(df_localizacao_final.info())

Total de 670 localidades únicas foram encontradas e carregadas.


,municipio,estado,uf_sigla,regiao,bairro,cep,nome_rua,numero_rua
0,ACRELANDIA,ACRE,AC,REGIÃO NORTE,CENTRO,69945-000,AVENIDA ADENILSON ROGERIO DE OLIVEIRA,72
1,CRUZEIRO DO SUL,ACRE,AC,REGIÃO NORTE,CENTRO,69980-000,MARGEM ESQUERDA DO RIO JURUA,S/N
2,RIO BRANCO,ACRE,AC,REGIÃO NORTE,AVIARIO,69909-720,AVENIDA NACOES UNIDAS,23
3,SENA MADUREIRA,ACRE,AC,REGIÃO NORTE,TRIANGULO,69940-000,AVENIDA BRASIL,2154
4,SENADOR GUIOMARD,ACRE,AC,REGIÃO NORTE,CENTRO,69925-000,AVENIDA CASTELO BRANCO,2709


,municipio,estado,uf_sigla,regiao,bairro,cep,nome_rua,numero_rua
665,GURUPI,TOCANTINS,TO,REGIÃO NORTE,SETOR CENTRAL,77410-010,AVENIDA GOIAS,2508
666,PALMAS,TOCANTINS,TO,REGIÃO NORTE,PLANO DIRETOR SUL,77020-126,"QUADRA 110 SUL (ARSE 14), AV. NS 08, H.M.",110
667,PARAISO DO TOCANTINS,TOCANTINS,TO,REGIÃO NORTE,CENTRO,77600-000,AVENIDA TRANSBRASILIANA,961
668,PONTE ALTA DO TOCANTINS,TOCANTINS,TO,REGIÃO NORTE,TAQUARUCU,77590-000,AVENIDA TIRADENTES QUADRA V,SN
669,PORTO NACIONAL,TOCANTINS,TO,REGIÃO NORTE,JARDIM BRASÍLIA,77500-000,AVENIDA ENGENHEIRO LUIZ CRUZ,S/N


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   municipio   670 non-null    object
 1   estado      670 non-null    object
 2   uf_sigla    670 non-null    object
 3   regiao      670 non-null    object
 4   bairro      670 non-null    object
 5   cep         670 non-null    object
 6   nome_rua    670 non-null    object
 7   numero_rua  670 non-null    object
dtypes: object(8)
memory usage: 42.0+ KB


None

## `dim_revendedor`
Neste etapa, vamos popular a tabela `dim_revendedor` extraindo, limpando e unificando dados dos revendedores.

In [9]:

arquivos_de_precos = [
    'ca-2004-01.csv', 'ca-2004-02.csv',
    'ca-2010-01.csv', 'ca-2010-02.csv',
    'ca-2016-01.csv', 'ca-2016-02.csv',
    'ca-2020-01.csv', 'ca-2020-02.csv',
    'ca-2022-01.csv', 'ca-2022-02.csv',
    'ca-2025-01.csv'
]

lista_dfs_revendedores = []


mapa_colunas_revendedor = {
    'CNPJ da Revenda': 'cnpj',
    'Revenda': 'revenda_nome',
    'Bandeira': 'bandeira'
}

for nome_arquivo in arquivos_de_precos:
    caminho_completo = os.path.join(PASTA_DADOS, nome_arquivo)
    print(f"Processando arquivo: {nome_arquivo}...")
    
    
    colunas_originais_no_arquivo = pd.read_csv(
        caminho_completo, 
        sep=';', 
        encoding='utf-8-sig', 
        nrows=0
        ).columns
    
    colunas_para_ler = [col for col in colunas_originais_no_arquivo if col in mapa_colunas_revendedor.keys()]
    
    if not colunas_para_ler:
        continue
    
    
    chunk_iterator = pd.read_csv(
        caminho_completo, sep=';', encoding='utf-8-sig', 
        usecols=colunas_para_ler, chunksize=50000, low_memory=False
    )
    
    for chunk in chunk_iterator:
        # Renomeando usando o mapa completo e adiciona à lista
        chunk.rename(columns=mapa_colunas_revendedor, inplace=True)
        # Limpando o CNPJ para conter apenas números
        chunk['cnpj'] = chunk['cnpj'].str.replace(r'\D', '', regex=True)
        
        lista_dfs_revendedores.append(chunk)


df_revendedores_bruto = pd.concat(lista_dfs_revendedores, ignore_index=True)
df_revendedores_bruto.dropna(subset=['cnpj'], inplace=True)
df_revendedor_final = df_revendedores_bruto.drop_duplicates(subset=['cnpj']).copy()

colunas_finais_revendedor = ['cnpj', 'revenda_nome', 'bandeira']

# Reordenamos o DataFrame para garantir a consistência
df_revendedor_final = df_revendedor_final[colunas_finais_revendedor]

print("\nExtração e limpeza final dos revendedores concluídas.")

Processando arquivo: ca-2004-01.csv...
Processando arquivo: ca-2004-02.csv...
Processando arquivo: ca-2010-01.csv...
Processando arquivo: ca-2010-02.csv...
Processando arquivo: ca-2016-01.csv...
Processando arquivo: ca-2016-02.csv...
Processando arquivo: ca-2020-01.csv...
Processando arquivo: ca-2020-02.csv...
Processando arquivo: ca-2022-01.csv...
Processando arquivo: ca-2022-02.csv...
Processando arquivo: ca-2025-01.csv...

Extração e limpeza final dos revendedores concluídas.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_revendedor_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `dim_revendedor` que criamos no MySQL.

In [10]:
# Carregando os dados para a tabela dim_revendedor
try:
    df_revendedor_final.to_sql('dim_revendedor', con=engine, if_exists='append', index=False)
    print(f"{len(df_revendedor_final)} registros únicos de revendedores carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

33143 registros únicos de revendedores carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [11]:
print(f"Total de {len(df_revendedor_final)} revendedores únicos foram encontrados e carregados.")
display(df_revendedor_final.head())
display(df_revendedor_final.tail())
display(df_revendedor_final.info())

Total de 33143 revendedores únicos foram encontrados e carregados.


,cnpj,revenda_nome,bandeira
0,49051667000102,AUTO POSTO SAKAMOTO LTDA,PETROBRAS DISTRIBUIDORA S.A.
3,00003188000121,COMPETRO COMERCIO E DISTRIBUICAO DE DERIVADOS ...,BRANCA
6,00603738000143,GASOL COMBUSTÍVEIS AUTOMOTIVOS LTDA.,PETROBRAS DISTRIBUIDORA S.A.
9,34274233001508,PETROBRAS DISTRIBUIDORA S.A.,PETROBRAS DISTRIBUIDORA S.A.
12,34274233003381,PETROBRAS DISTRIBUIDORA S.A.,PETROBRAS DISTRIBUIDORA S.A.


,cnpj,revenda_nome,bandeira
5492941,26232032000106,AUTO POSTO ESTRADA DOS ROMEIROS LTDA,VIBRA
5493175,33211167000150,ROMEIROS 1 COMERCIO DE PETROLEO LTDA,VIBRA
5493307,33210833000136,AUTO POSTO ROMEIROS 5 LTDA,VIBRA
5493661,34481410000113,W & L COMERCIO DE COMBUSTIVEL LTDA,IPIRANGA
5493672,45855959000147,AUTO POSTO BRAZ LTDA,IPIRANGA


<class 'pandas.core.frame.DataFrame'>
Index: 33143 entries, 0 to 5493672
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cnpj          33143 non-null  object
 1   revenda_nome  33143 non-null  object
 2   bandeira      33142 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


None

## `dim_produto`
Neste etapa, vamos popular a tabela `dim_produto` extraindo, limpando e unificando dados de multiplos arquivos de origem.

In [12]:
# Lista de arquivos ATUALIZADA para corresponder exatamente à sua pasta 'Data'
arquivos_fonte = [
    'producao-petroleo-m3-1997-2025.csv',
    'processamento-petroleo-m3-1990-2025.csv',
    'importacoes-exportacoes-petroleo-2000-2025.csv',
    'importacoes-exportacoes-derivados-2000-2025.csv',
    'ca-2004-01.csv', 'ca-2004-02.csv',
    'ca-2010-01.csv', 'ca-2010-02.csv',
    'ca-2016-01.csv', 'ca-2016-02.csv',
    'ca-2020-01.csv', 'ca-2020-02.csv',
    'ca-2022-01.csv', 'ca-2022-02.csv',
    'ca-2025-01.csv'
]

# Usando um 'set' para guardar os nomes únicos de forma eficiente
produtos_unicos = set()

# Iterando sobre cada arquivo para coletar os produtos
for nome_arquivo in arquivos_fonte:
    caminho_completo = os.path.join(PASTA_DADOS, nome_arquivo)
    
    # Verifica se o arquivo existe antes de tentar processar
    if not os.path.exists(caminho_completo):
        print(f"AVISO: Arquivo não encontrado, pulando: {nome_arquivo}")
        continue
        
    print(f"Processando arquivo: {nome_arquivo}...")
    try:
        colunas_no_arquivo = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig', nrows=0).columns
        
        # Mantendo sua lógica robusta para checar as colunas 'PRODUTO' e 'MATÉRIA PRIMA'
        colunas_a_processar = []
        if 'PRODUTO' in colunas_no_arquivo: colunas_a_processar.append('PRODUTO')
        if 'Produto' in colunas_no_arquivo: colunas_a_processar.append('Produto') # Adicionando variação
        if 'MATÉRIA PRIMA' in colunas_no_arquivo: colunas_a_processar.append('MATÉRIA PRIMA')
            
        for nome_coluna in colunas_a_processar:
            chunk_iterator = pd.read_csv(
                caminho_completo, sep=';', encoding='utf--8-sig',
                usecols=[nome_coluna], chunksize=50000, low_memory=False
            )
            for chunk in chunk_iterator:
                produtos_unicos.update(chunk[nome_coluna].dropna().unique())

    except Exception as e:
        print(f"  --> Erro ao processar o arquivo {nome_arquivo}: {e}")

print(f"\nColeta finalizada. {len(produtos_unicos)} nomes de produtos únicos encontrados.")

Processando arquivo: producao-petroleo-m3-1997-2025.csv...
Processando arquivo: processamento-petroleo-m3-1990-2025.csv...
Processando arquivo: importacoes-exportacoes-petroleo-2000-2025.csv...
Processando arquivo: importacoes-exportacoes-derivados-2000-2025.csv...
Processando arquivo: ca-2004-01.csv...
Processando arquivo: ca-2004-02.csv...
Processando arquivo: ca-2010-01.csv...
Processando arquivo: ca-2010-02.csv...
Processando arquivo: ca-2016-01.csv...
Processando arquivo: ca-2016-02.csv...
Processando arquivo: ca-2020-01.csv...
Processando arquivo: ca-2020-02.csv...
Processando arquivo: ca-2022-01.csv...
Processando arquivo: ca-2022-02.csv...
Processando arquivo: ca-2025-01.csv...

Coleta finalizada. 26 nomes de produtos únicos encontrados.


In [13]:
print("Iniciando a transformação e enriquecimento dos produtos coletados...")

# Criando o DataFrame final a partir do 'set' que já está na memória
lista_final_produtos = sorted(list(produtos_unicos))
df_produto_final = pd.DataFrame(lista_final_produtos, columns=['produto_nome'])

# Definindo nossa função de classificação
def classificar_produto_final(produto_nome):
    produto_nome_upper = str(produto_nome).upper()
    categoria = 'Outros'
    subcategoria = 'N/A'

    if 'GASOLINA' in produto_nome_upper:
        categoria = 'Gasolina'
        if 'DE AVIAÇÃO' in produto_nome_upper: subcategoria = 'Gasolina de Aviação'
        else: subcategoria = 'Gasolina A'
    elif 'ÓLEO DIESEL' in produto_nome_upper:
        categoria = 'Óleo Diesel'
    elif 'ETANOL' in produto_nome_upper:
        categoria = 'Etanol'
    elif 'QUEROSENE' in produto_nome_upper:
        categoria = 'Querosene'
        if 'DE AVIAÇÃO' in produto_nome_upper: subcategoria = 'Querosene de Aviação'
        else: subcategoria = 'Querosene Iluminante'
    elif 'COMBUSTÍVEIS PARA AERONAVES' in produto_nome_upper:
        categoria = 'Combustíveis de Aviação'
    elif 'COMBUSTÍVEIS PARA NAVIOS' in produto_nome_upper or 'ÓLEO COMBUSTÍVEL' in produto_nome_upper:
        categoria = 'Combustíveis Navais'
    elif 'PETRÓLEO' in produto_nome_upper:
        categoria = 'Petróleo Cru'
        if 'IMPORTADO' in produto_nome_upper: subcategoria = 'Importado'
        elif 'NACIONAL' in produto_nome_upper: subcategoria = 'Nacional'
    elif 'GLP' in produto_nome_upper:
        categoria = 'GLP'
    elif 'GNV' in produto_nome_upper:
        categoria = 'GNV'
    elif 'OUTRAS CARGAS' in produto_nome_upper:
        categoria = 'Cargas Genéricas'
    elif produto_nome_upper in ['ASFALTO', 'COQUE', 'LUBRIFICANTE', 'NAFTA', 'PARAFINA', 'SOLVENTE']:
        categoria = 'Derivado Industrial/Especial'
        subcategoria = produto_nome.capitalize()
    elif 'OUTROS NÃO ENERGÉTICOS' in produto_nome_upper:
        categoria = 'Não Energéticos'
        
    return categoria, subcategoria

# Aplicando a função para criar as novas colunas
df_produto_final[['categoria', 'subcategoria']] = df_produto_final['produto_nome'].apply(
    lambda nome: pd.Series(classificar_produto_final(nome))
)

# Organizando as colunas na ordem final
colunas_finais_produto = ['produto_nome', 'categoria', 'subcategoria']
df_produto_final = df_produto_final[colunas_finais_produto]

Iniciando a transformação e enriquecimento dos produtos coletados...


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_produto_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `dim_produto` que criamos no MySQL.

In [14]:
try:
    df_produto_final.to_sql('dim_produto', con=engine, if_exists='append', index=False)
    print(f"{len(df_produto_final)} registros únicos de produtos carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

26 registros únicos de produtos carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [15]:
print(f"Total de {len(df_produto_final)} produtos únicos foram encontrados e carregados.")
display(df_produto_final.head())
display(df_produto_final.tail())
display(df_produto_final.info())

Total de 26 produtos únicos foram encontrados e carregados.


,produto_nome,categoria,subcategoria
0,ASFALTO,Derivado Industrial/Especial,Asfalto
1,COMBUSTÍVEIS PARA AERONAVES,Combustíveis de Aviação,N/A
2,COMBUSTÍVEIS PARA NAVIOS,Combustíveis Navais,N/A
3,COQUE,Derivado Industrial/Especial,Coque
4,DIESEL,Outros,N/A


,produto_nome,categoria,subcategoria
21,QUEROSENE DE AVIAÇÃO,Querosene,Querosene de Aviação
22,QUEROSENE ILUMINANTE,Querosene,Querosene Iluminante
23,SOLVENTE,Derivado Industrial/Especial,Solvente
24,ÓLEO COMBUSTÍVEL,Combustíveis Navais,N/A
25,ÓLEO DIESEL,Óleo Diesel,N/A


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   produto_nome  26 non-null     object
 1   categoria     26 non-null     object
 2   subcategoria  26 non-null     object
dtypes: object(3)
memory usage: 756.0+ bytes


None

## `dim_refinaria`
Neste etapa, vamos popular a tabela `dim_refinaria` extraindo, limpando e unificando dados de multiplos arquivos de origem.

In [16]:
# Definindo o arquivo fonte e as colunas de interesse
arquivo_fonte_refinaria = 'processamento-petroleo-m3-1990-2025.csv'
caminho_refinaria = os.path.join(PASTA_DADOS, arquivo_fonte_refinaria)
colunas_refinaria = ['REFINARIA', 'UNIDADE DA FEDERAÇÃO']

# Lendo os dados brutos
df_refinarias_bruto = pd.read_csv(caminho_refinaria, sep=';', encoding='utf-8-sig', usecols=colunas_refinaria)

# Renomeando e limpando
df_refinarias_bruto.rename(columns={'REFINARIA': 'nome_refinaria', 'UNIDADE DA FEDERAÇÃO': 'estado'}, inplace=True)
df_refinarias_bruto.dropna(subset=['nome_refinaria'], inplace=True)
df_refinarias_final = df_refinarias_bruto.drop_duplicates().copy()

# Buscando dados da dim_localizacao para usar como tabela de consulta (lookup).
print("Buscando dados da dim_localizacao para enriquecimento...")
df_lookup_local = pd.read_sql('SELECT id_localizacao, estado FROM dim_localizacao', engine)

# Como um estado pode ter vários municípios, removemos as duplicatas de estado 
# para garantir que cada estado tenha um único id_localizacao para o JOIN.
df_lookup_local_estados = df_lookup_local.drop_duplicates(subset=['estado']).copy()

# Cruzando (JOIN) nossa lista de refinarias com a tabela de localização para obter o id_localizacao
print("Enriquecendo dados da refinaria com id_localizacao...")
df_refinaria_para_carga = pd.merge(
    df_refinarias_final,
    df_lookup_local_estados,
    on='estado',
    how='left' # Usamos 'left' para manter todas as refinarias
)

# Selecionando e reordenando as colunas finais para a carga no banco de dados
df_refinaria_para_carga = df_refinaria_para_carga[['nome_refinaria', 'id_localizacao']]

print("Extração e transformação das refinarias concluídas.")

Buscando dados da dim_localizacao para enriquecimento...
Enriquecendo dados da refinaria com id_localizacao...
Extração e transformação das refinarias concluídas.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_refinaria_para_carga` limpo e pronto, o próximo passo é carregá-lo para a tabela `dim_refinaria` que criamos no MySQL.

In [17]:
try:
    df_refinaria_para_carga.to_sql('dim_refinaria', con=engine, if_exists='append', index=False)
    print(f"{len(df_refinaria_para_carga)} registros únicos de refinaria carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

21 registros únicos de refinaria carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [18]:
print(f"Total de {len(df_refinaria_para_carga)} refinarias únicas foram encontrados e carregados.")
display(df_refinaria_para_carga.head())
display(df_refinaria_para_carga.tail())
display(df_refinaria_para_carga.info())

Total de 21 refinarias únicas foram encontrados e carregados.


,nome_refinaria,id_localizacao
0,DAX OIL,32
1,RPBC,538
2,UNIVEN,538
3,RLAM,32
4,REDUC,374


,nome_refinaria,id_localizacao
16,LUBNOR,80
17,REAM,22
18,3R POTIGUAR (ex-RPCC),417
19,REFMAT,32
20,SSOIL,538


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nome_refinaria  21 non-null     object
 1   id_localizacao  21 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 468.0+ bytes


None

# **Tabelas Fato**

A partir desta etapa, iremos começar o ETL das tabelas fatos, onde vai exigir o uso de lookups para consultas e merge para fazer os joins entre tabelas utilizando as FKs.

## `f_producao`
Neste etapa, vamos popular a tabela `f_producao`, utilizando FKs de id_calendario, id_localizacao e id_produto. Obtendo, também, dados da produção relacionado a origem da produção do petróleo (MAR OU TERRA) e volume produzido.

In [19]:

# para o calendario vamos usar por convenção o dia 1 de cada mês

dim_calendario_lookup = pd.read_sql("SELECT id_calendario, ano, mes_numero FROM dim_calendario WHERE dia_numero = 1", engine)
dim_localizacao_lookup = pd.read_sql("SELECT id_localizacao, estado FROM dim_localizacao", engine)
dim_localizacao_lookup_estados = dim_localizacao_lookup.drop_duplicates(subset=['estado']).copy()
dim_produto_lookup = pd.read_sql("SELECT id_produto, produto_nome FROM dim_produto", engine)

print("Dimensões de consultas prontas.")


print("\nIniciando ETL para a tabela f_producao...")

arquivo_producao = 'producao-petroleo-m3-1997-2025.csv'
caminho_producao = os.path.join(PASTA_DADOS, arquivo_producao)
df_producao_bruto = pd.read_csv(caminho_producao, sep=';', encoding='utf-8-sig')

mapa_mes_para_num = {
    'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12
}


# Renomeando e limpando os dados
df_producao_bruto.rename(columns={

    'ANO': 'ano', 'MÊS': 'mes_nome', 'UNIDADE DA FEDERAÇÃO': 'estado',
    'PRODUTO': 'produto_nome', 'PRODUÇÃO': 'volume_produzido',
    'LOCALIZAÇÃO': 'tipo_origem_producao', 'GRANDE REGIÃO': 'regiao'

}, inplace=True)


# Corrigindo a ',' nos volumes
df_producao_bruto['volume_produzido'] = df_producao_bruto['volume_produzido'].astype(str)
df_producao_bruto['volume_produzido'] = df_producao_bruto['volume_produzido'].str.replace(',', '.', regex=False)


df_producao_bruto['volume_produzido'] = pd.to_numeric(df_producao_bruto['volume_produzido'], errors='coerce')
df_producao_bruto.dropna(subset=['volume_produzido'], inplace=True)
df_producao_bruto['mes_numero'] = df_producao_bruto['mes_nome'].str.upper().map(mapa_mes_para_num)



# Fazendo os JOINs (pd.merge) para encontrar os IDs (FKs)
df_com_ids = pd.merge(df_producao_bruto, dim_calendario_lookup, on=['ano', 'mes_numero'], how='left')
df_com_ids = pd.merge(df_com_ids, dim_localizacao_lookup_estados, on='estado', how='left')
df_com_ids = pd.merge(df_com_ids, dim_produto_lookup, on='produto_nome', how='left')


# Selecionando apenas as colunas que irão para a tabela Fato
colunas_fato = [
    'id_calendario', 'id_localizacao', 'id_produto',
    'tipo_origem_producao', 'volume_produzido'
]



# Definindo a produção final sem valores nulos
df_producao_final = df_com_ids[colunas_fato].dropna().copy()
print(f"\nTransformação da 'f_producao' concluída! {len(df_producao_final)} registros prontos para a carga.")

Dimensões de consultas prontas.

Iniciando ETL para a tabela f_producao...

Transformação da 'f_producao' concluída! 7656 registros prontos para a carga.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_producao_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `f_producao` que criamos no MySQL.

In [20]:
try:
    df_producao_final.to_sql('f_producao', con=engine, if_exists='append', index=False)
    print(f"{len(df_producao_final)} registros únicos de revendedores carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

7656 registros únicos de revendedores carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [21]:
print(f"Total de {len(df_producao_final)} revendedores únicos foram encontrados e carregados.")
display(df_producao_final.head())
display(df_producao_final.tail())
display(df_producao_final.info())

Total de 7656 revendedores únicos foram encontrados e carregados.


,id_calendario,id_localizacao,id_produto,tipo_origem_producao,volume_produzido
0,19970101,22,19,TERRA,122200.0
1,19970201,22,19,TERRA,124785.0
2,19970301,22,19,TERRA,128177.0
3,19970401,22,19,TERRA,124968.0
4,19970501,22,19,TERRA,115710.0


,id_calendario,id_localizacao,id_produto,tipo_origem_producao,volume_produzido
7651,20250801,260,19,MAR,0.0
7652,20250901,260,19,MAR,0.0
7653,20251001,260,19,MAR,0.0
7654,20251101,260,19,MAR,0.0
7655,20251201,260,19,MAR,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656 entries, 0 to 7655
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_calendario         7656 non-null   int64  
 1   id_localizacao        7656 non-null   int64  
 2   id_produto            7656 non-null   int64  
 3   tipo_origem_producao  7656 non-null   object 
 4   volume_produzido      7656 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 299.2+ KB


None

## `f_processamento`
Neste etapa, vamos popular a tabela `f_processamento`, utilizando FKs de id_calendario, id_refinaria e id_produto. Obtendo, também, dados do processamento relacionado ao volume processado.

In [22]:
### 1. Preparação: Carregando as Dimensões para Lookup

print("Carregando dimensões para a memória para usar como lookup...")

# Carregamos as dimensões necessárias para esta tabela Fato
dim_calendario_lookup = pd.read_sql("SELECT id_calendario, ano, mes_numero FROM dim_calendario WHERE dia_numero = 1", engine)
dim_produto_lookup = pd.read_sql("SELECT id_produto, produto_nome FROM dim_produto", engine)
dim_refinaria_lookup = pd.read_sql("SELECT id_refinaria, nome_refinaria FROM dim_refinaria", engine)

print("Dimensões de lookup prontas.")


### 2. Extração e Transformação da f_processamento

print("\nIniciando o ETL para a tabela f_processamento...")

# Lendo o arquivo inteiro de uma vez
arquivo_processamento = 'processamento-petroleo-m3-1990-2025.csv'
caminho_processamento = os.path.join(PASTA_DADOS, arquivo_processamento)
df_processamento_bruto = pd.read_csv(caminho_processamento, sep=';', encoding='utf-8-sig')

# Mapa para converter nome do mês em número
mapa_mes_para_num = {
    'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12
}

# Renomeando e limpando os dados
df_processamento_bruto.rename(columns={
    'ANO': 'ano', 'MÊS': 'mes_nome', 'REFINARIA': 'nome_refinaria',
    'MATÉRIA PRIMA': 'produto_nome', 'PROCESSADO': 'volume_processado'
}, inplace=True)

df_processamento_bruto['volume_processado'] = pd.to_numeric(df_processamento_bruto['volume_processado'], errors='coerce')
df_processamento_bruto.dropna(subset=['volume_processado'], inplace=True)
df_processamento_bruto['mes_numero'] = df_processamento_bruto['mes_nome'].str.upper().map(mapa_mes_para_num)

# Fazendo os JOINs (pd.merge) para encontrar os IDs (FKs)
df_com_ids = pd.merge(df_processamento_bruto, dim_calendario_lookup, on=['ano', 'mes_numero'], how='left')
df_com_ids = pd.merge(df_com_ids, dim_produto_lookup, on='produto_nome', how='left')
df_com_ids = pd.merge(df_com_ids, dim_refinaria_lookup, on='nome_refinaria', how='left')

# Selecionando apenas as colunas que irão para a tabela Fato
colunas_fato = [
    'id_calendario', 'id_produto', 'id_refinaria', 'volume_processado'
]
df_processamento_final = df_com_ids[colunas_fato].dropna().copy()

# Convertendo os IDs para inteiros para garantir a consistência
df_processamento_final['id_calendario'] = df_processamento_final['id_calendario'].astype(int)
df_processamento_final['id_produto'] = df_processamento_final['id_produto'].astype(int)
df_processamento_final['id_refinaria'] = df_processamento_final['id_refinaria'].astype(int)


print(f"\nTransformação da 'f_processamento' concluída! {len(df_processamento_final)} registros prontos para a carga.")

Carregando dimensões para a memória para usar como lookup...
Dimensões de lookup prontas.

Iniciando o ETL para a tabela f_processamento...

Transformação da 'f_processamento' concluída! 22671 registros prontos para a carga.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_processamento_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `f_processamento` que criamos no MySQL.

In [23]:
try:
    df_processamento_final.to_sql('f_processamento', con=engine, if_exists='append', index=False)
    print(f"{len(df_processamento_final)} registros únicos de processamento carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

22671 registros únicos de processamento carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [24]:
print(f"Total de {len(df_processamento_final)} registros únicos foram encontrados e carregados.")
display(df_processamento_final.head())
display(df_processamento_final.tail())
display(df_processamento_final.info())

Total de 22671 registros únicos foram encontrados e carregados.


,id_calendario,id_produto,id_refinaria,volume_processado
0,19901201,16,1,0
1,19900901,20,1,0
2,19901001,20,1,0
3,19901101,20,1,0
4,19901201,20,1,0


,id_calendario,id_produto,id_refinaria,volume_processado
22666,20250401,21,21,799
22667,20250301,21,21,0
22668,20250201,21,21,0
22669,20250501,20,2,136
22670,20250601,16,21,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22671 entries, 0 to 22670
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   id_calendario      22671 non-null  int64
 1   id_produto         22671 non-null  int64
 2   id_refinaria       22671 non-null  int64
 3   volume_processado  22671 non-null  int64
dtypes: int64(4)
memory usage: 708.6 KB


None

## `f_comercio_exterior`
Neste etapa, vamos popular a tabela `f_comercio_exterior`, utilizando FKs de id_produto e id_calendario. Obtendo, também, dados de importação/exportação de petróleo e seus derivados, relacionado ao volume negociado, valor negociado e tipo de operação (IMPORTAÇÃO OU EXPORTAÇÃO).

In [25]:
dim_produto_lookup = pd.read_sql("SELECT id_produto, produto_nome FROM dim_produto", engine)
dim_calendario_lookup = pd.read_sql("SELECT id_calendario, ano, mes_numero FROM dim_calendario WHERE dia_numero = 1", engine)
print("Dimensões de consultas prontas.")


print("\nIniciando o ETL para a tabela f_comercio_exterior...")

# --- INÍCIO DA MUDANÇA ---
# 1. Lista com os dois arquivos de comércio exterior
arquivos_comercio = [
    'importacoes-exportacoes-derivados-2000-2025.csv',
    'importacoes-exportacoes-petroleo-2000-2025.csv'
]
lista_dfs_comercio = [] # Lista para armazenar os DataFrames processados

# O mapa de meses que já utilizamos
mapa_mes_para_num = {
    'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12
}

# 2. Loop para processar cada arquivo individualmente
for nome_arquivo in arquivos_comercio:
    print(f"Processando o arquivo: {nome_arquivo}...")
    caminho_comercio = os.path.join(PASTA_DADOS, nome_arquivo)
    df_comercio_bruto = pd.read_csv(caminho_comercio, sep=';', encoding='utf-8-sig')
    
    df_comercio_bruto.rename(columns={
        'ANO': 'ano', 
        'MÊS': 'mes_nome', 
        'PRODUTO': 'produto_nome',
        'OPERAÇÃO COMERCIAL': 'tipo_operacao',
        'IMPORTADO / EXPORTADO': 'volume_negociado',
        'DISPÊNDIO / RECEITA': 'valor_negociado'
    }, inplace=True)

    df_comercio_bruto['volume_negociado'] = df_comercio_bruto['volume_negociado'].astype(str)
    df_comercio_bruto['volume_negociado'] = df_comercio_bruto['volume_negociado'].str.replace(',', '.', regex=False)
    
    df_comercio_bruto['valor_negociado'] = df_comercio_bruto['valor_negociado'].astype(str)
    df_comercio_bruto['valor_negociado'] = df_comercio_bruto['valor_negociado'].str.replace(',', '.', regex=False)
    
    df_comercio_bruto['volume_negociado'] = pd.to_numeric(df_comercio_bruto['volume_negociado'], errors='coerce')
    df_comercio_bruto['valor_negociado'] = pd.to_numeric(df_comercio_bruto['valor_negociado'], errors='coerce')
    
    df_comercio_bruto['mes_numero'] = df_comercio_bruto['mes_nome'].str.upper().map(mapa_mes_para_num)
    
    df_comercio_bruto.dropna(subset=['volume_negociado', 'valor_negociado'], inplace=True)
    
    # Adiciona o DataFrame processado à nossa lista
    lista_dfs_comercio.append(df_comercio_bruto)

# 3. Unificar todos os DataFrames da lista em um só
print("\nUnificando os dados dos dois arquivos...")
df_comercio_unificado = pd.concat(lista_dfs_comercio, ignore_index=True)
print(f"Arquivos unificados com sucesso! Total de {len(df_comercio_unificado)} registros brutos.")
# --- FIM DA MUDANÇA ---


# Fazendo os JOINs (pd.merge) no DataFrame unificado para encontrar os IDs (FKs)
df_com_ids = pd.merge(df_comercio_unificado, dim_calendario_lookup, on=['ano', 'mes_numero'], how='left')
df_com_ids = pd.merge(df_com_ids, dim_produto_lookup, on='produto_nome', how='left')


# Selecionando apenas as colunas que irão para a tabela Fato
colunas_fato = [
    'id_calendario', 
    'id_produto',
    'tipo_operacao',
    'volume_negociado',
    'valor_negociado'
]

# Definindo o dataframe final, sem valores nulos nas chaves
df_comercio_final = df_com_ids[colunas_fato].dropna().copy()

print(f"\nTransformação da 'f_comercio_exterior' concluída! {len(df_comercio_final)} registros prontos para a carga.")

Dimensões de consultas prontas.

Iniciando o ETL para a tabela f_comercio_exterior...
Processando o arquivo: importacoes-exportacoes-derivados-2000-2025.csv...
Processando o arquivo: importacoes-exportacoes-petroleo-2000-2025.csv...

Unificando os dados dos dois arquivos...
Arquivos unificados com sucesso! Total de 9806 registros brutos.

Transformação da 'f_comercio_exterior' concluída! 9806 registros prontos para a carga.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_comercio_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `f_comercio_exterior` que criamos no MySQL.

In [26]:
try:
    df_comercio_final.to_sql('f_comercio_exterior', con=engine, if_exists='append', index=False)
    print(f"{len(df_comercio_final)} registros únicos de comércio carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

9806 registros únicos de comércio carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [27]:
print(f"Total de {len(df_comercio_final)} registros únicos foram encontrados e carregados.")
display(df_comercio_final.head())
display(df_comercio_final.tail())
display(df_comercio_final.info())

Total de 9806 registros únicos foram encontrados e carregados.


,id_calendario,id_produto,tipo_operacao,volume_negociado,valor_negociado
0,20000401,1,EXPORTAÇÃO,1362.035577,309443
1,20000801,1,EXPORTAÇÃO,2700.793269,544917
2,20001201,1,EXPORTAÇÃO,1839.723077,426760
3,20000201,1,EXPORTAÇÃO,1676.491346,270633
4,20000101,1,EXPORTAÇÃO,1702.567308,360648


,id_calendario,id_produto,tipo_operacao,volume_negociado,valor_negociado
9801,20250101,19,IMPORTAÇÃO,1289574.654,633572724
9802,20250701,19,IMPORTAÇÃO,1154166.107,533184316
9803,20250601,19,IMPORTAÇÃO,1255895.879,547698306
9804,20250501,19,IMPORTAÇÃO,1173650.256,527925790
9805,20250301,19,IMPORTAÇÃO,1220041.255,602886781


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9806 entries, 0 to 9805
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_calendario     9806 non-null   int64  
 1   id_produto        9806 non-null   int64  
 2   tipo_operacao     9806 non-null   object 
 3   volume_negociado  9806 non-null   float64
 4   valor_negociado   9806 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 383.2+ KB


None

## `f_precos_combustiveis`
Neste etapa, vamos popular a tabela `f_precos_combustiveis`, utilizando FKs de id_produto, id_calendario, id_revendedor e id_localizacao. Obtendo, também, dados dos combustiveis automotivos relacionados ao valor de venda, valor de compra e unidade de medida(R$/Litros e R$/M³).

In [28]:
dim_calendario_lookup = pd.read_sql("SELECT id_calendario, data_completa FROM dim_calendario", engine)
dim_produto_lookup = pd.read_sql("SELECT id_produto, produto_nome FROM dim_produto", engine)
dim_revendedor_lookup = pd.read_sql("SELECT id_revendedor, cnpj FROM dim_revendedor", engine) # Removido revenda_nome para otimizar
dim_localizacao_lookup = pd.read_sql("SELECT id_localizacao, municipio, uf_sigla FROM dim_localizacao", engine) # Removido estado para otimizar
print("Dimensões de consultas prontas.")


print("\nIniciando o ETL para a tabela f_precos_combustiveis...")

# Lista de todos os arquivos de preços de combustíveis (ATUALIZADA)
arquivos_precos = [
    'ca-2004-01.csv', 'ca-2004-02.csv',
    'ca-2010-01.csv', 'ca-2010-02.csv',
    'ca-2016-01.csv', 'ca-2016-02.csv',
    'ca-2020-01.csv', 'ca-2020-02.csv',
    'ca-2022-01.csv', 'ca-2022-02.csv',
    'ca-2025-01.csv'
]

lista_dfs_precos = []

# Dicionário de mapeamento de colunas para padronização
mapa_colunas = {
    'Regiao - Sigla': 'regiao_sigla',
    'Estado - Sigla': 'uf_sigla',
    'Municipio': 'municipio',
    'Revenda': 'revenda_nome',
    'CNPJ da Revenda': 'cnpj',
    'Nome da Rua': 'nome_rua',
    'Numero Rua': 'numero_rua',
    'Complemento': 'complemento',
    'Bairro': 'bairro',
    'Cep': 'cep',
    'Produto': 'produto_nome',
    'Data da Coleta': 'data_completa',
    'Valor de Venda': 'valor_venda',
    'Valor de Compra': 'valor_compra',
    'Unidade de Medida': 'unidade_medida',
    'Bandeira': 'bandeira'
}


# Loop principal para processar os arquivos em pedaços (chunks)
for nome_arquivo in arquivos_precos:
    caminho_completo = os.path.join(PASTA_DADOS, nome_arquivo)
    
    if not os.path.exists(caminho_completo):
        print(f"AVISO: Arquivo não encontrado, pulando: {nome_arquivo}")
        continue
        
    print(f"Processando arquivo em chunks: {nome_arquivo}...")
    
    chunk_iterator = pd.read_csv(
        caminho_completo,
        sep=';',
        encoding='utf-8-sig',
        usecols=mapa_colunas.keys(),
        chunksize=50000,
        low_memory=False
    )
    
    for chunk in chunk_iterator:
        chunk.rename(columns=mapa_colunas, inplace=True)
        
      
        
        chunk['cnpj'] = chunk['cnpj'].str.replace(r'\D', '', regex=True)
        
        chunk['data_completa'] = pd.to_datetime(chunk['data_completa'], dayfirst=True, errors='coerce')
        
        chunk['valor_venda'] = pd.to_numeric(chunk['valor_venda'].astype(str).str.replace(',', '.', regex=False), errors='coerce')
        chunk['valor_compra'] = pd.to_numeric(chunk['valor_compra'].astype(str).str.replace(',', '.', regex=False), errors='coerce')
        
        chunk.dropna(subset=['data_completa', 'valor_venda', 'cnpj'], inplace=True)
        
        lista_dfs_precos.append(chunk)

# Unificando todos os chunks
df_precos_unificado = pd.concat(lista_dfs_precos, ignore_index=True)
print(f"\nDados unificados! {len(df_precos_unificado)} registros brutos prontos para os JOINs.")


print("\nIniciando os JOINs com as dimensões...")

dim_calendario_lookup['data_completa'] = pd.to_datetime(dim_calendario_lookup['data_completa'])

df_com_ids = pd.merge(df_precos_unificado, dim_calendario_lookup, on='data_completa', how='left')
df_com_ids = pd.merge(df_com_ids, dim_produto_lookup, on='produto_nome', how='left')
df_com_ids = pd.merge(df_com_ids, dim_revendedor_lookup, on='cnpj', how='left')
df_com_ids = pd.merge(df_com_ids, dim_localizacao_lookup, on=['municipio', 'uf_sigla'], how='left')

print("JOINs concluídos!")

# Selecionando apenas as colunas que irão para a tabela Fato
colunas_fato = [
    'id_calendario',
    'id_produto',
    'id_revendedor',
    'id_localizacao',
    'valor_venda',
    'valor_compra',
    'unidade_medida'
]


id_cols = ['id_calendario', 'id_produto', 'id_revendedor', 'id_localizacao']
df_precos_final = df_com_ids[colunas_fato].dropna(subset=id_cols).copy()

# Convertendo FKs para inteiros para garantir consistência
for col in ['id_calendario', 'id_produto', 'id_revendedor', 'id_localizacao']:
    df_precos_final[col] = df_precos_final[col].astype(int)

print(f"\nTransformação da 'f_precos_combustiveis' concluída! {len(df_precos_final)} registros prontos para a carga.")

Dimensões de consultas prontas.

Iniciando o ETL para a tabela f_precos_combustiveis...
Processando arquivo em chunks: ca-2004-01.csv...
Processando arquivo em chunks: ca-2004-02.csv...
Processando arquivo em chunks: ca-2010-01.csv...
Processando arquivo em chunks: ca-2010-02.csv...
Processando arquivo em chunks: ca-2016-01.csv...
Processando arquivo em chunks: ca-2016-02.csv...
Processando arquivo em chunks: ca-2020-01.csv...
Processando arquivo em chunks: ca-2020-02.csv...
Processando arquivo em chunks: ca-2022-01.csv...
Processando arquivo em chunks: ca-2022-02.csv...
Processando arquivo em chunks: ca-2025-01.csv...

Dados unificados! 5493769 registros brutos prontos para os JOINs.

Iniciando os JOINs com as dimensões...
JOINs concluídos!

Transformação da 'f_precos_combustiveis' concluída! 5493769 registros prontos para a carga.


### Carregando os Dados para o Banco de Dados
Com nosso DataFrame `df_precos_final` limpo e pronto, o próximo passo é carregá-lo para a tabela `f_precos_combustiveis` que criamos no MySQL.

In [29]:
try:
    df_precos_final.to_sql('f_precos_combustiveis', con=engine, if_exists='append', index=False, method='multi', chunksize=20000)
    print(f"{len(df_precos_final)} registros únicos dos preços de combustiveis carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a carga: {e}")

5493769 registros únicos dos preços de combustiveis carregados com sucesso!


### Verificação
Vamos visualizar o início e o fim do DataFrame final que foi carregado

In [30]:
print(f"Total de {len(df_precos_final)} preços de combustiveis únicos foram encontrados e carregados.")
display(df_precos_final.head())
display(df_precos_final.tail())
display(df_precos_final.info())

Total de 5493769 preços de combustiveis únicos foram encontrados e carregados.


,id_calendario,id_produto,id_revendedor,id_localizacao,valor_venda,valor_compra,unidade_medida
0,20040511,8,1,581,1.967,1.6623,R$ / litro
1,20040511,7,1,581,0.899,0.6282,R$ / litro
2,20040511,5,1,581,1.299,1.1704,R$ / litro
3,20040510,8,2,650,1.850,1.6700,R$ / litro
4,20040510,7,2,650,0.780,0.4800,R$ / litro


,id_calendario,id_produto,id_revendedor,id_localizacao,valor_venda,valor_compra,unidade_medida
5493764,20250630,7,32978,73,4.65,NaN,R$ / litro
5493765,20250630,8,32946,427,6.05,NaN,R$ / litro
5493766,20250630,10,32946,427,6.15,NaN,R$ / litro
5493767,20250630,6,32946,427,5.59,NaN,R$ / litro
5493768,20250630,7,32946,427,4.39,NaN,R$ / litro


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5493769 entries, 0 to 5493768
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id_calendario   int64  
 1   id_produto      int64  
 2   id_revendedor   int64  
 3   id_localizacao  int64  
 4   valor_venda     float64
 5   valor_compra    float64
 6   unidade_medida  object 
dtypes: float64(2), int64(4), object(1)
memory usage: 293.4+ MB


None